# Init

In [1]:
# from skfem import *
import numpy as np
from utils import *
from skfem.helpers import d, dd, ddd, dot, ddot, grad, dddot, prod
from scipy.sparse.linalg import LinearOperator, minres
# from skfem.models.poisson import *
from skfem.assembly import BilinearForm, LinearForm
import datetime
import pandas as pd
import sys
import time


tol = 1e-8
intorder = 6
solver_type = 'mgcg'
refine_time = 7
element_type = 'P1'
sigma = 5 
penalty = False
epsilon = 1e-6
ep = epsilon
example = 'ex3'
save_path = 'log/' + example + '_' + element_type + '_' + ('pen' if penalty else 'nopen') + '_' +'{}'.format(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

## Save $u_{h0}$ with different $h$ 

In [2]:
# # %%time
# for i in range(1, 10):
#     m = MeshTri()
#     base_order = i
#     base_path = 'uh0_{}.npy'.format(base_order)
#     m.refine(base_order)

#     if penalty:
#         uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)
#     else:
#         uh0, basis = solve_problem1(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)

#     np.save(base_path, uh0)
#     print('{} th saved'.format(i))

## Reading previous solution

In [3]:
test_order = 3
test_path = 'uh0_{}.npy'.format(test_order)
m = MeshTri()
m.refine(test_order)
test_basis, test_fbasis = solve_problem2(m, element_type, intorder=3, basis_only=True)
test_uh0 = np.load(test_path)

In [4]:
base_order = 7
base_path = 'uh0_{}.npy'.format(base_order)
m = MeshTri()
m.refine(base_order)
base_basis, base_fbasis = solve_problem2(m, element_type, intorder=3, basis_only=True)
base_uh0 = np.load(base_path)

In [8]:
test_basis['u'].basis[0]

(DiscreteField(value=array([[0.55555556, 0.44      , 0.44      , 0.68      ],
        [0.22222222, 0.16      , 0.12      , 0.44      ],
        [0.22222222, 0.12      , 0.16      , 0.44      ],
        [0.22222222, 0.16      , 0.12      , 0.44      ],
        [0.22222222, 0.12      , 0.16      , 0.44      ],
        [0.55555556, 0.44      , 0.44      , 0.68      ],
        [0.22222222, 0.16      , 0.12      , 0.44      ],
        [0.55555556, 0.44      , 0.44      , 0.68      ],
        [0.22222222, 0.12      , 0.16      , 0.44      ],
        [0.22222222, 0.16      , 0.12      , 0.44      ],
        [0.55555556, 0.44      , 0.44      , 0.68      ],
        [0.22222222, 0.16      , 0.12      , 0.44      ],
        [0.55555556, 0.44      , 0.44      , 0.68      ],
        [0.22222222, 0.12      , 0.16      , 0.44      ],
        [0.22222222, 0.16      , 0.12      , 0.44      ],
        [0.22222222, 0.12      , 0.16      , 0.44      ],
        [0.22222222, 0.12      , 0.16      , 0.44   

In [11]:
m = MeshTri()
m.refine(3)
base_uh0, base_basis = solve_problem1(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)
m = MeshTri()
m.refine(5)
test_uh0, test_basis = solve_problem1(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)

In [12]:
project(test_uh0, basis_from=test_basis['u'], basis_to=base_basis['u'])

ValueError: could not broadcast input array from shape (128) into shape (2048)

## projection

In [5]:
uh0_project = project(test_uh0, basis_from=test_basis['u'], basis_to=base_basis['u'])

ValueError: could not broadcast input array from shape (32768) into shape (128)

In [ ]:
basis_to = InteriorBasis(m, ElementTriMorley(), intorder=3)
# test_basis['u']

In [ ]:
uh0_project = project(test_uh0, basis_to, base_basis['u'])

In [ ]:
basis_store['u'].interpolator(uh0_store)(np.array([[x[i][j]], [y[i][j]]]))

In [ ]:
# compute errors
U = basis['u'].interpolate(uh0).value

L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
Du = get_DuError(basis['u'], uh0)
H1u = Du + L2u

if penalty:
    D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
else:
    D2u = get_D2uError(basis['u'], uh0)

epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)

In [ ]:
uh0_store = uh0
basis_store = basis

In [ ]:
%%time
m = MeshTri()
m.refine(3)

if penalty:
    uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)
else:
    uh0, basis = solve_problem1(m, element_type, solver_type, intorder=3, tol=1e-8, epsilon=1e-6)

U = basis['u'].interpolate(uh0).value

# compute errors

L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
Du = get_DuError(basis['u'], uh0)
H1u = Du + L2u

if penalty:
    D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
else:
    D2u = get_D2uError(basis['u'], uh0)

epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)

In [ ]:
L2u

In [ ]:
def exact_u_n(x, y):
    return basis_store['u'].interpolator(uh0_store)(np.array([[x], [y]]))

In [ ]:
@Functional
def L2uError_h(w):
    x, y = w.x
    return (w.w - un)**2

In [ ]:
np.sqrt(L2uError_h.assemble(basis['u'], w=U))

In [ ]:
np.sqrt(L2uError_h.assemble(basis['u'], w=U))

In [ ]:
x, y = www.x
exact_u(x, y)

In [ ]:
from tqdm import tqdm

In [ ]:
un = np.zeros_like(x)
for i in tqdm(range(x.shape[0])):
    for j in range(x.shape[1]):
        un[i][j] = basis_store['u'].interpolator(uh0_store)(np.array([[x[i][j]], [y[i][j]]]))

In [ ]:
uh0.shape

In [ ]:
un

In [ ]:
U.shape

In [ ]:
uh0.shape

In [ ]:
basis['u'].interpolator(uh0)(np.array([[0.5],[0.5]]))

In [ ]:
m = MeshTri()
# m.refine(1)

if element_type == 'P1':
    element = {'w': ElementTriP1(), 'u': ElementTriMorley()}
elif element_type == 'P2':
    element = {'w': ElementTriP2(), 'u': ElementTriMorley()}
else:
    raise Exception("The element not supported")

basis = {
    variable: InteriorBasis(m, e, intorder=3)
    for variable, e in element.items()
}

In [ ]:
# basis['u'].dx # weights
refine2 = basis['u'].global_coordinates().value

In [ ]:
basis['u'].interpolator(uh0)(np.array([[0.5],[0.5]]))

# Solving

In [ ]:
# solving 

print('=======Arguments=======')
print('penalty:\t{}'.format(penalty))
print('element_type:\t{}'.format(element_type))
print('solver_type:\t{}'.format(solver_type))
print('tol:\t{}'.format(tol))
print('intorder:\t{}'.format(intorder))
print('refine_time:\t{}'.format(refine_time))
print('epsilon:\t{}'.format(epsilon))
print('sigma:\t{}'.format(sigma))
print('=======Results=======')

time_start = time.time()

ep = epsilon
df_list = []
L2_list = []
Du_list = []
D2u_list = []
h_list = []
epu_list = []
m = MeshTri()

for i in range(1, refine_time+1):

    m.refine()

    if penalty:
        uh0, basis, fbasis = solve_problem2(m, element_type, solver_type, intorder=6, tol=1e-8, epsilon=1e-6)
    else:
        uh0, basis = solve_problem1(m, element_type, solver_type, intorder=6, tol=1e-8, epsilon=1e-6)

    U = basis['u'].interpolate(uh0).value

    # compute errors

    L2u = np.sqrt(L2uError.assemble(basis['u'], w=U))
    Du = get_DuError(basis['u'], uh0)
    H1u = Du + L2u
    
    if penalty:
        D2u = np.sqrt(get_D2uError(basis['u'], uh0)**2 + L2pnvError.assemble(fbasis, w=fbasis.interpolate(uh0)))
    else:
        D2u = get_D2uError(basis['u'], uh0)
        
    epu = np.sqrt(epsilon**2 * D2u**2 + Du**2)
    h_list.append(m.param())
    Du_list.append(Du)
    L2_list.append(L2u)
    D2u_list.append(D2u)
    epu_list.append(epu)

hs = np.array(h_list)
L2s = np.array(L2_list)
Dus = np.array(Du_list)
D2us = np.array(D2u_list)
epus = np.array(epu_list)
H1s = L2s + Dus
H2s = H1s + D2us

# store data
data = np.array([L2s, H1s, H2s, epus])
df = pd.DataFrame(data.T, columns=['L2', 'H1', 'H2', 'Energy'])
df_list.append(df)

show_result(L2s, H1s, H2s, epus)

# df.to_csv(save_path+'.csv')
time_end = time.time()
print('Total Time Cost {:.2f} s'.format(time_end-time_start))